In [2]:
import numpy as np

In [3]:
# read in the text, transforming everything to lower case
text_org = open('Beiras.txt').read().lower()
print('our original text has ' + str(len(text_org)) + ' characters')


our original text has 589161 characters


In [4]:
print(text_org[:1000])

 [http://resistiromorir.blogspot.com.es/p/xose-manuel-beiras-discurso-na.html]
 ben se decatarían xa as suas señorías de que o texto que veño de ler non está tirado de ningunha copla panfletaria contra as leoninas taxas impostas polo ministro de xustiza actual, malia que vulneran frontalmente o dereito dos cidadáns do común a unha xustiza gratuita, e reduce a cinzas a norma contida no artigo 24.1 da constitución, que predica que "tódalas persoas teñen dereito a obteren a tutela efectiva dos xuíces e tribunais no exercicio dos seus dereitos e intereses lexítimos, sen que, en ningún caso, se poida producir indefensión". porque resulta evidente que esa reaccionária decisión ministerial incurre na aberración de convirtir o recurso dos cidadáns á xustiza nunha mercaduría, un ben que compre mercar con cartos, so pena de ficar en indefensión.       non, señorías, non. ese texto non procede de ningún panfleto popular anónimo. tampouco é produto da malsán e rebirichada imaxinación do beiras. lo

In [5]:
import re
text_without_source="";
regexp=re.compile(r'http')
for line in text_org.splitlines():
    if not regexp.search(line):
        text_without_source= text_without_source + line
    

In [6]:
chars=sorted(list(set(text_without_source)))
print(chars);

[' ', '!', '"', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¡', 'ª', '«', '´', 'º', '»', '¿', 'à', 'á', 'â', 'ä', 'ç', 'è', 'é', 'ê', 'ì', 'í', 'ï', 'ñ', 'ó', 'ô', 'ö', 'ú', 'ü', '–', '—', '‘', '’', '“', '”', '•', '…']


In [7]:

text_clean = re.sub('[ºªàâäçèêïìôöü]|\W+',' ',text_without_source)
text_clean = text_clean.replace("  "," ")
chars=sorted(list(set(text_clean )))
print(chars);
print ("this corpus has " +  str(len(text_clean)) + " total number of characters")
print ("this corpus has " +  str(len(chars)) + " unique characters")


[' ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'á', 'é', 'í', 'ñ', 'ó', 'ú']
this corpus has 568575 total number of characters
this corpus has 44 unique characters


In [8]:
print(text_clean[:1000])

 ben se decatarían xa as suas señorías de que o texto que veño de ler non está tirado de ningunha copla panfletaria contra as leoninas taxas impostas polo ministro de xustiza actual malia que vulneran frontalmente o dereito dos cidadáns do común a unha xustiza gratuita e reduce a cinzas a norma contida no artigo 24 1 da constitución que predica que tódalas persoas teñen dereito a obteren a tutela efectiva dos xuíces e tribunais no exercicio dos seus dereitos e intereses lexítimos sen que en ningún caso se poida producir indefensión porque resulta evidente que esa reaccionária decisión ministerial incurre na aberración de convirtir o recurso dos cidadáns á xustiza nunha mercaduría un ben que compre mercar con cartos so pena de ficar en indefensión non señorías non ese texto non procede de ningún panfleto popular anónimo tampouco é produto da malsán e rebirichada imaxinación do beiras lonxe diso ben se decatarían xa as suas cultas señorías de que é a reprodución literal dun punxente poem

In [9]:
def window_transform_text(text,window_size,step_size):
    # containers for input/output pairs
    inputs = []
    outputs = []
    #Number of windows to create
    n_windows=int((len(text) - window_size)/ step_size)
    for j in range(n_windows) :
        # k .- Start index
        k= j * step_size
        inputs.append(text[k:(k+window_size)])
        outputs.append(text[k+window_size])

    return inputs,outputs

In [10]:
# run your text window-ing function 
window_size = 100
step_size = 5
inputs, outputs = window_transform_text(text_clean,window_size,step_size)

In [11]:

# print out a few of the input/output pairs to verify that we've made the right kind of stuff to learn from
print('input = ' + inputs[2])
print('output = ' + outputs[2])
print('--------------')
print('input = ' + inputs[100])
print('output = ' + outputs[100])

input = catarían xa as suas señorías de que o texto que veño de ler non está tirado de ningunha copla panfle
output = t
--------------
input = ún caso se poida producir indefensión porque resulta evidente que esa reaccionária decisión minister
output = i


In [12]:
# this dictionary is a function mapping each unique character to a unique integer
chars_to_indices = dict((c, i) for i, c in enumerate(chars))  # map each unique character to unique integer

# this dictionary is a function mapping each unique integer back to a unique character
indices_to_chars = dict((i, c) for i, c in enumerate(chars))  # map each unique integer back to unique character

In [13]:
# transform character-based input/output into equivalent numerical versions
def encode_io_pairs(text,window_size,step_size):
    # number of unique chars
    chars = sorted(list(set(text)))
    num_chars = len(chars)
    
    # cut up text into character input/output pairs
    inputs, outputs = window_transform_text(text,window_size,step_size)
    
    # create empty vessels for one-hot encoded input/output
    X = np.zeros((len(inputs), window_size, num_chars), dtype=np.bool)
    y = np.zeros((len(inputs), num_chars), dtype=np.bool)
    
    # loop over inputs/outputs and tranform and store in X/y
    for i, sentence in enumerate(inputs):
        for t, char in enumerate(sentence):
            X[i, t, chars_to_indices[char]] = 1
        y[i, chars_to_indices[outputs[i]]] = 1
        
    return X,y

In [14]:
# use your function
window_size = 100
step_size = 5
X,y = encode_io_pairs(text_clean,window_size,step_size)

In [15]:
### necessary functions from the keras library
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import keras
import random

# TODO build the required RNN model: a single LSTM hidden layer with softmax activation, categorical_crossentropy loss 
#Number of unique chars
num_chars = len(chars)

model= Sequential()
# 1 Layer .- LSTM layer 1 should be an LSTM module with 200 hidden units
model.add(LSTM(200,input_shape = (window_size,num_chars)))
# 2 Layer .-  Dense, with number chars unit and softmax activation
model.add(Dense(num_chars,activation='softmax'))


# initialize optimizer
optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

# compile model --> make sure initialized optimizer and callbacks - as defined above - are used
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


Using TensorFlow backend.


In [16]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/cpu:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 11909648797020002359, name: "/gpu:0"
 device_type: "GPU"
 memory_limit: 103809024
 locality {
   bus_id: 1
 }
 incarnation: 5733125414780857672
 physical_device_desc: "device: 0, name: GRID K520, pci bus id: 0000:00:03.0"]

In [17]:
Xsmall = X[:10000,:,:]
ysmall = y[:10000,:]

In [19]:
# train the model
model.fit(Xsmall, ysmall, batch_size=500, epochs=40,verbose = 1)

# save weights
model.save_weights('model_weights/best_RNN_small_textdata_weights.hdf5')

Epoch 1/40
10000/10000 [==============================] - 6s - loss: 1.8807     
Epoch 2/40
10000/10000 [==============================] - 6s - loss: 1.8661     
Epoch 3/40
10000/10000 [==============================] - 6s - loss: 1.8473     
Epoch 4/40
10000/10000 [==============================] - 6s - loss: 1.8224     
Epoch 5/40
10000/10000 [==============================] - 6s - loss: 1.8062     
Epoch 6/40
10000/10000 [==============================] - 6s - loss: 1.7884     
Epoch 7/40
10000/10000 [==============================] - 6s - loss: 1.7641     
Epoch 8/40
10000/10000 [==============================] - 6s - loss: 1.7491     
Epoch 9/40
10000/10000 [==============================] - 6s - loss: 1.7247     
Epoch 10/40
10000/10000 [==============================] - 6s - loss: 1.7019     
Epoch 11/40
10000/10000 [==============================] - 6s - loss: 1.6757     
Epoch 12/40
10000/10000 [==============================] - 6s - loss: 1.6538     
Epoch 13/40
10000/10000 [

ImportError: `save_weights` requires h5py.